<a href="https://colab.research.google.com/github/Shk-tech/my-colab-learning-material/blob/main/2507_korea_pytorch_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#fashin-mnist -> deep learning
#mnist -> sklearn....

In [ ]:
import torch
import torchvision # pytorch에서 제공해주는.. vision library... opencv... PIL...
import matplotlib.pyplot as plt
from torch import nn

In [ ]:
mnist_train = torchvision.datasets.FashionMNIST(root='.', train=True, download=True)
mnist_valid = torchvision.datasets.FashionMNIST(root='.', train=False, download=True)

100%|██████████| 26.4M/26.4M [00:01<00:00, 18.9MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 304kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.57MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 7.20MB/s]


In [ ]:
x_train = mnist_train.data / 255
y_train = mnist_train.targets
x_valid = mnist_valid.data / 255
y_valid = mnist_valid.targets
x_train.shape, y_train.shape, x_train.dtype, y_train.dtype

(torch.Size([60000, 28, 28]), torch.Size([60000]), torch.float32, torch.int64)

In [ ]:
#PyTorch에서 이진 분류 문제를 풀 때
# target : 0~1, float32,  .... (N, 1)
y_train_5 = (y_train == 0).float().reshape(-1, 1)
y_valid_5 = (y_valid == 0).float().reshape(-1, 1)

In [ ]:
dataset_train = list(zip(x_train, y_train_5))
dataset_valid = list(zip(x_valid, y_valid_5))

loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True, drop_last=True)
loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=32)

for x, y in loader_train:
    break
x.shape, y.shape, x.dtype, y.dtype

(torch.Size([32, 28, 28]), torch.Size([32, 1]), torch.float32, torch.float32)

In [ ]:
flat = nn.Flatten()
flat(x).shape

torch.Size([32, 784])

In [ ]:
model = nn.Sequential(
    nn.Flatten(),# (N,x) reshape(?, -1)

    nn.Linear(28*28, 128),
    nn.BatchNorm1d(128),
    nn.SiLU(), # 비선형 - Tanh, ReLU

    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.SiLU(),

    nn.Linear(64, 1),
    nn.Sigmoid()
)

opt = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()
 # loss함수의 요구사항 pred - (확률, logit), shape
 #                   true - (확률, float), shape
prob = model(x)
loss_fn(prob, y)

# 입력데이터 X, Y, 모델의 출력 f(X)->Y^
# shape, min/max, dtype
# -> loss_fn이 요구하는 y^, Y를 제공해줘야 한다.

tensor(0.7533, grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
# 오버피딩 방지 3종세트
# 1. Regularization (모델의 표현력 저하)
# 2. Dropout   (BatchNorm을 못 쓸 때... 온라인 학습.. MLP)
# 3. BatchNorm (!!)

In [ ]:
# BatchNorm을 쓸 때 주의점
# 1. batch-size (너무작으면 안 된다. 16)
# 2. random..
# 3. 배치학습 (온라인x)
# 4. train/eval
# -> 무조건 쓸 수록 학습속도.성능

In [ ]:
# #####################################
# import cv2
# cam = cv2.VideoCamera(0)
# while True:
#     image, ret = cam.read()
#     image = preproc(image)
#     model.eval()
#     with torch.no_grad():
#         logit = model(image)


#     if logit > 0:
#         stop() # PLC
#     else:
#         continue

In [ ]:
for epoch in range(20):
    model.train()
    t_loss = 0
    t_acc = 0
    for step, (x, y) in enumerate(loader_train):
        #1. feed-forwarding   2. loss   3. grad   4. update
        prob = model(x)
        loss = loss_fn(prob, y * 0.8 + 0.1) # 0 or 1 -> 0.1 or 0.9
        opt.zero_grad()
        loss.backward()
        opt.step()

        #metrics
        t_loss += loss.item()
        pred = (prob.detach() > 0.5).float()
        t_acc += (y == pred).float().mean()
        print("\r e=%d s=%d l=%.4f a=%.4f" % (epoch, step, t_loss/(step+1), t_acc/(step+1)), end='')
    print()

    model.eval()
    t_loss = 0
    t_acc = 0
    for step, (x, y) in enumerate(loader_valid):
        #1. feed-forwarding   2. loss
        with torch.no_grad():
            prob = model(x)
        loss = loss_fn(prob, y)

        #metrics
        t_loss += loss.item()
        pred = (prob.detach() > 0.5).float()
        t_acc += (y == pred).float().mean()
        print("\r%s s=%d l=%.4f a=%.4f" % (' '*50, step, t_loss/(step+1), t_acc/(step+1)), end='')
    print()

 e=0 s=1874 l=0.3816 a=0.9556
                                                   s=312 l=0.1813 a=0.9590
 e=1 s=1874 l=0.3698 a=0.9625
                                                   s=312 l=0.1827 a=0.9643
 e=2 s=1874 l=0.3662 a=0.9658
                                                   s=312 l=0.1696 a=0.9634
 e=3 s=1874 l=0.3645 a=0.9678
                                                   s=312 l=0.1729 a=0.9658
 e=4 s=1874 l=0.3620 a=0.9710
                                                   s=312 l=0.1742 a=0.9675
 e=5 s=1874 l=0.3603 a=0.9718
                                                   s=312 l=0.1791 a=0.9656
 e=6 s=1874 l=0.3590 a=0.9729
                                                   s=312 l=0.1655 a=0.9678
 e=7 s=1874 l=0.3575 a=0.9743
                                                   s=312 l=0.1630 a=0.9698
 e=8 s=1874 l=0.3566 a=0.9753
                                                   s=312 l=0.1613 a=0.9709
 e=9 s=1874 l=0.3555 a=0.9768
                         

In [ ]:
a = ['a', 'b', 'c']
b = ['d', 'e', 'f']
for index, (ai, bi) in enumerate(zip(a, b)):
    print(ai, bi)

a d
b e
c f


In [ ]:
d = nn.Dropout(0.3)
dummy = torch.arange(10).float()
dummy
d.eval()
d(dummy)

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

tensor([ 0.0000,  1.2500,  2.5000,  3.7500,  5.0000,  0.0000,  7.5000,  8.7500,
        10.0000, 11.2500])

In [ ]:
import numpy as np